In [1]:
import boto3
import pandas as pd
import datetime
import io

In [2]:
from nanoHUB.application import Application
application = Application.get_instance()

nanohub_metrics_db = application.new_db_engine('nanohub_metrics')

nanoHUB - Serving Students, Researchers & Instructors


In [3]:
bucket_name = 'nanoHUB-analytics'
endpoint_url = 'https://' + application.get_config_value('geddesapi.endpoint') + ':443'

boto_session = boto3.session.Session(
    aws_access_key_id = application.get_config_value('geddesapi.access_key'),
    aws_secret_access_key = application.get_config_value('geddesapi.secret_key'),
)

s3_client1 = boto_session.client(
    service_name='s3',
    endpoint_url = endpoint_url
) 


In [4]:
response = s3_client1.list_buckets()
print(response)

{'ResponseMetadata': {'RequestId': 'tx00000000000000002b976-00619344f8-a6d52f-my-store', 'HostId': '', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 16 Nov 2021 05:43:20 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amz-request-id': 'tx00000000000000002b976-00619344f8-a6d52f-my-store', 'strict-transport-security': 'max-age=15724800; includeSubDomains'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'db2.nanohub', 'CreationDate': datetime.datetime(2021, 11, 14, 7, 19, 33, 62000, tzinfo=tzlocal())}, {'Name': 'db2.nanohub-metrics.toolevents', 'CreationDate': datetime.datetime(2021, 11, 14, 7, 32, 19, 930000, tzinfo=tzlocal())}, {'Name': 'db2.nanohub-metrics.toolstart', 'CreationDate': datetime.datetime(2021, 11, 14, 7, 32, 19, 659000, tzinfo=tzlocal())}, {'Name': 'nanohub.raw', 'CreationDate': datetime.datetime(2021, 11, 14, 16, 3, 18, 337000, tzinfo=tzlocal())}], 'Owner': {'DisplayName': 'Praveen Saxena', 'ID': 'saxenap'}}


In [5]:
#bucket = s3_client1.create_bucket(Bucket='db2.nanohub-metrics.toolstart')
#bucket = s3_client1.create_bucket(Bucket='db2.nanohub-metrics.toolevents')
#bucket = s3_client1.create_bucket(Bucket='nanohub.raw')
bucket = s3_client1.create_bucket(Bucket='nanohub.processed')

#print(bucket)
display(s3_client1.list_buckets())

{'ResponseMetadata': {'RequestId': 'tx00000000000000002b978-00619344fa-a6d52f-my-store',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 16 Nov 2021 05:43:22 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amz-request-id': 'tx00000000000000002b978-00619344fa-a6d52f-my-store',
   'strict-transport-security': 'max-age=15724800; includeSubDomains'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'db2.nanohub',
   'CreationDate': datetime.datetime(2021, 11, 14, 7, 19, 33, 62000, tzinfo=tzlocal())},
  {'Name': 'db2.nanohub-metrics.toolevents',
   'CreationDate': datetime.datetime(2021, 11, 14, 7, 32, 19, 930000, tzinfo=tzlocal())},
  {'Name': 'db2.nanohub-metrics.toolstart',
   'CreationDate': datetime.datetime(2021, 11, 14, 7, 32, 19, 659000, tzinfo=tzlocal())},
  {'Name': 'nanohub.processed',
   'CreationDate': datetime.datetime(2021, 11, 16, 5, 43, 21, 844000, tzinfo=tzlocal())},
  {'Name': 'nanohu

In [10]:
for key in s3_client1.list_objects(Bucket='nanohub.raw')['Contents']:
    print(key['Key'])
    obj = s3_client1.get_object(Bucket='nanohub.raw', Key=key['Key'])
    
    display(pd.read_parquet(io.BytesIO(obj['Body'].read())))
    

KeyError: 'Contents'

In [16]:
key = 'nanohub_metrics/toolevents/2019-07-09.parquet.gzip'
obj = s3_client1.get_object(Bucket='nanohub.raw', Key=key)
display(pd.read_parquet(io.BytesIO(obj['Body'].read())))



,entryID,source,job,superjob,sessnum,event,start,finish,walltime,cputime,...,ip,host,domain,countryShort,countryLong,region,city,latitude,longitude,recStatus
0,19051338,from joblog,0,0,1500017,application,2019-07-08 00:00:37,2019-07-09 01:09:15,4102.0,151.993,...,209.95.56.53,209.95.56.53.static.midphase.com,midphase.com,US,United States,Utah,Providence,41.700370,-111.806791,1
1,19050473,from joblog,0,0,1500018,application,2019-07-08 00:01:22,2019-07-09 00:06:27,287.0,6.586,...,23.243.56.228,cpe-23-243-56-228.socal.res.rr.com,rr.com,US,United States,California,Santa Monica,34.019450,-118.491190,1
2,19050472,from joblog,0,0,1500019,application,2019-07-08 00:01:31,2019-07-09 00:06:24,277.0,3.139,...,23.243.56.228,cpe-23-243-56-228.socal.res.rr.com,rr.com,US,United States,California,Santa Monica,34.019450,-118.491190,1
3,19032177,from joblog,7274395,0,1499923,:mail2self:,2019-07-08 00:03:03,2019-07-08 00:03:32,0.0,0.080,...,128.46.19.134,scatter.nanohub.org,nanohub.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
4,19032423,from joblog,7274396,0,1499923,:mail2self:,2019-07-08 00:04:05,2019-07-08 00:04:07,1.0,0.070,...,128.46.19.134,scatter.nanohub.org,nanohub.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145639,19050206,from joblog,7277631,0,1499923,:mail2self:,2019-07-08 23:49:45,2019-07-08 23:50:03,0.0,0.040,...,128.46.19.134,scatter.nanohub.org,nanohub.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
145640,19050212,from joblog,7277632,0,1498881,[waiting],2019-07-08 23:51:22,2019-07-08 23:56:02,192.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
145641,19050213,from joblog,7277632,0,1498881,/probegridsite.sh,2019-07-08 23:51:22,2019-07-08 23:56:02,0.0,0.010,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
145642,19050210,from joblog,7277633,0,1498881,[waiting],2019-07-08 23:52:42,2019-07-08 23:56:01,110.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1


In [11]:
for key in s3_client1.list_objects(Bucket='nanohub.raw')['Contents']:
    print(key['Key'])
    s3_client1.delete_object(Bucket='nanohub.raw', Key=key['Key'])

KeyError: 'Contents'

In [14]:
sql_query = '''
SELECT * FROM nanohub_metrics.toolevents 
    WHERE start >='2012-12-25 00:00:00'
    AND start <'2012-12-26 00:00:00'
;
'''


df = pd.read_sql_query(sql_query, nanohub_metrics_db)
display(df.head())
display(df.tail())
display(df.info())

,entryID,source,job,superjob,sessnum,event,start,finish,walltime,cputime,...,ip,host,domain,countryShort,countryLong,region,city,latitude,longitude,recStatus
0,6410036,from joblog,0,0,553089,application,2012-12-25 00:01:21,2012-12-25 00:01:21,0.0,1.664,...,218.2.216.14,unknown,unknown,CN,China,Jiangsu,Nanjing,32.061670,118.777780,1
1,6482224,from joblog,2705861,0,546612,simulation,2012-12-25 00:01:47,2012-12-25 00:01:47,0.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
2,6482225,from joblog,2705861,0,546612,[waiting],2012-12-25 00:01:47,2012-12-25 00:02:51,64.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
3,6409771,from joblog,0,0,553090,application,2012-12-25 00:02:18,2012-12-25 02:18:55,8197.0,0.992,...,85.90.101.194,pulsarnpp.ru,pulsarnpp.ru,RU,Russian Federation,Moskva,Moscow,55.752220,37.615560,1
4,6482226,from joblog,2705862,0,546612,simulation,2012-12-25 00:02:19,2012-12-25 00:02:20,1.0,0.020,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1


,entryID,source,job,superjob,sessnum,event,start,finish,walltime,cputime,...,ip,host,domain,countryShort,countryLong,region,city,latitude,longitude,recStatus
2863,6484803,from joblog,2707453,0,546612,simulation,2012-12-25 23:58:48,2012-12-25 23:58:48,0.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
2864,6484804,from joblog,2707453,0,546612,[waiting],2012-12-25 23:58:48,2012-12-26 00:12:47,839.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
2865,6484805,from joblog,2707454,0,546612,simulation,2012-12-25 23:58:57,2012-12-25 23:58:57,0.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
2866,6484806,from joblog,2707454,0,546612,[waiting],2012-12-25 23:58:57,2012-12-26 00:02:43,226.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
2867,6410042,from joblog,0,0,553171,application,2012-12-25 23:59:13,2012-12-26 00:24:49,1536.0,1.319,...,120.194.82.142,unknown,unknown,CN,China,Henan,Zhumadian,32.979440,114.029440,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2868 entries, 0 to 2867
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   entryID       2868 non-null   int64         
 1   source        2868 non-null   object        
 2   job           2868 non-null   int64         
 3   superjob      2868 non-null   int64         
 4   sessnum       2868 non-null   int64         
 5   event         2868 non-null   object        
 6   start         2868 non-null   datetime64[ns]
 7   finish        2868 non-null   datetime64[ns]
 8   walltime      2868 non-null   float64       
 9   cputime       2868 non-null   float64       
 10  viewtime      2868 non-null   float64       
 11  user          2868 non-null   object        
 12  tool          2868 non-null   object        
 13  protocol      2868 non-null   int64         
 14  success       2868 non-null   int64         
 15  status        2868 non-null   int64   

None

In [ ]:
start = datetime.date(2012,12,25)
end = datetime.date.today()
end = datetime.date(2012,12,26)



#while start < end:
nextday = start + datetime.timedelta(days = 1)
print(start, nextday, end)
sql_query_daily = "select * from nanohub_metrics.toolevents where start >= '%s 00:00:00' and start < '%s 00:00:00'" % (start, nextday)
print(sql_query_daily)

df = pd.read_sql_query(sql_query_daily, nanohub_metrics_db)
display(df.head())
display(df.tail())
display(df.info())

start = nextday